<a href="https://colab.research.google.com/github/jorgefdzlara/NLP_EQ17/blob/main/Equipo17_semana09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Procesamiento de Lenguaje Natural (NLP)**
### Tecnológico de Monterrey
### Prof Luis Eduardo Falcón Morales

### **Nombres y matrículas de los integrantes del equipo:**
* Diego Alonso Luna Ramirez  - A01793035
* Jessica Lechuga Ramos - A01793512
* José Javier Granados Hernández - A0056717
* Jorge Fernández Lara - A01793062
* Kevin Brandon Cruz Mejía -A01794176

## **Adtividad de la Semana 09**
### **Traducciones y Clasificación de Textos.**

## Instrucciones
En la actividad de esta semana trabajarás en equipos mediante el uso de Transformers.

HuggingFace (HF) es una plataforma de una comunidad de IA que promueve las contribuciones de código abierto, en particular para compartir modelos de aprendizaje profundo, entre ellos los Transformers o modelos basados en el mecanismo de Attention: https://huggingface.co/. La plataforma de HF está muy bien documentada, es de acceso abierto, pero deberás generar una cuenta sin costo para su uso. En ocasiones deberás generar una clave para el uso de algunos modelos, pero es solo para el control del uso de dicha plataforma y no generan algún costo para su uso educativo o de investigación, al menos hasta ahora. En particular en esta actividad estarás usando varios de los modelos Transformer que hacen tareas diversas, tanto en inglés como en español. El objetivo es que te familiarices con dicha plataforma y experimentes posteriormente con ella para las tareas o problemáticas particulares que te interesen.

In [ ]:
!pip install transformers sentencepiece sacremoses torch
!pip install transformers[torch]
!pip install datasets
!pip install accelerate -U
!pip install evaluate

                                              0.0/81.4 kB ? eta -:--:--
     ------------------------------           61.4/81.4 kB 1.1 MB/s eta 0:00:01
     ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00


In [ ]:
#Carga de librerías
import evaluate
import pandas as pd
import numpy as np
import torch

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AutoModelForSeq2SeqLM, pipeline, DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import accuracy_score, recall_score, f1_score

#### 1. Descarga el archivo amazonbaby5000.csv que se encuentra en Canvas. Este archivo está formado de comentarios en inglés sobre productos para bebé adquiridos en la plataforma de Amazon.

In [ ]:
#Carga de datos
df = pd.read_csv('amazonbaby5000.csv', sep=',', encoding='utf-8')

print('Total de registros de Amazon:',df.shape)

Total de registros de Amazon: (5000, 2)


In [ ]:
#Datos
df

,review,rating
0,I think it is more Expensive than drugstore th...,0
1,"When I saw this on Amazon, I put it into my wi...",1
2,We really like these valances. They have such...,1
3,No light emits from the night light. They pain...,0
4,I was really hoping for this to be a conventie...,0
...,...,...
4995,I like that this carrier is like the Moby in h...,1
4996,The box was damaged upon arrival. I was afraid...,1
4997,Purchased for graduation. Rec'd in 2 days like...,1
4998,For all of the reviews that said this car seat...,0


In [ ]:
X = df.review     # Serie de strings
Y = df.rating      # Serie de enteros 0s y 1s

#### 2. Realiza una partición de los datos en el porcentaje que consideres adecuado, entrenamiento, validación y prueba.

In [ ]:
#Acotar las oraciones
new_x = []
for i in range(len(X)):
  data = X [i]
  if len(X[i]) > 512:
    data = X[i][:512]
  new_x.append(data)

In [ ]:
#Creación del nuevo dataframe
new_df = df.copy()
new_df.columns = ['review', 'label']
new_df["review"] = new_x
del new_x

In [ ]:
# Conjunto de entrenamiento
train_data = new_df.sample(frac=0.8, random_state=40)

# Conjunto de prueba
temp = new_df.drop(train_data.index)

val_data = temp.sample(frac=0.5, random_state=40)

test_data = temp.drop(val_data.index)

print(f'El conjunto de entrenamiento tiene {len(train_data)} registros.')
print(f'El conjunto de validación tiene {len(val_data)} registos.')
print(f'El conjunto de prueba tiene {len(test_data)} registos.')

El conjunto de entrenamiento tiene 4000 registros.
El conjunto de validación tiene 500 registos.
El conjunto de prueba tiene 500 registos.


#### 3. Utiliza un modelo Transformer de HuggingFace para análisis de sentimiento en inglés y lleva a cabo la predicción de los comentarios en los siguientes casos:
- Utilizando un modelo predeterminado de HF de manera directa, es decir, sin ajuste de parámetros del modelo. Indicar el desempeño (accuracy) obtenido.
- Utiliza ahora un ajuste de parámetros (fine-tuning) con el modelo Transformer DistilBERT, para generar un nuevo modelo y clasificación de los comentarios. Es decir, el objetivo es utilizar un modelo Transformer pre-entrenado y adaptarlo (fine-tuning) o ajustarlo a las características de tus datos de entrenamiento. Indicar desempeño (accuracy) obtenido.
- Compara los resultados e incluye tus comentarios.


NOTA: Puedes apoyarte en el siguiente tutorial de la misma comunidad de HF:
https://huggingface.co/blog/sentiment-analysis-python

#### a) Utilizando un modelo predeterminado de HF de manera directa, es decir, sin ajuste de parámetros del modelo. Indicar el desempeño (accuracy) obtenido.

In [ ]:
def get_divace():
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        # Realiza operaciones en el dispositivo MPS
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        # Realiza operaciones en la GPU sin MPS
    else:
        device = torch.device("cpu")
    return device

In [ ]:
#Carga del modelo
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
device = get_divace()
model = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name, device=device)


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1+cpu)
    Python  3.11.3 (you have 3.11.1)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [ ]:
#Obtención de los valores de predicción de la conjunto de validación
predictions = model(test_data["review"].tolist())

predictions[:5]

[{'label': 'positive', 'score': 0.7387759685516357},
 {'label': 'negative', 'score': 0.8893738389015198},
 {'label': 'neutral', 'score': 0.5015902519226074},
 {'label': 'positive', 'score': 0.8450530767440796},
 {'label': 'positive', 'score': 0.7986863255500793}]

In [ ]:
def compute_metrics(y_pred, y_true):
    # Calcular la métrica f1
    accuracy = accuracy_score(y_true, y_pred)

    # Calcular el recall
    recall = recall_score(y_true, y_pred)

    # Calcular el F1-score
    f1 = f1_score(y_true, y_pred)

    return {"accuracy": accuracy, "f1": f1, "recall": recall}

In [ ]:
#Metricas aplicadas al resultado del modelo
lael_class = {"positive": 1, "neutral": 2, "negative": 0}
label_pred = []
label_true = test_data["label"].tolist()
for i in range(len(predictions)):
    label = lael_class[predictions[i]["label"]]
    if label == 2:
        label = 0 if label_true[i] == 1 else 1
    label_pred.append(label)

compute_metrics(label_pred, label_true)

{'accuracy': 0.724, 'f1': 0.8077994428969358, 'recall': 0.7435897435897436}

#### b) Utiliza ahora un ajuste de parámetros (fine-tuning) con el modelo Transformer DistilBERT, para generar un nuevo modelo y clasificación de los comentarios. Es decir, el objetivo es utilizar un modelo Transformer pre-entrenado y adaptarlo (fine-tuning) o ajustarlo a las características de tus datos de entrenamiento. Indicar desempeño (accuracy) obtenido.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    # Cargar métrica f1
    metric_f1 = evaluate.load("f1")

    # Calcular la métrica f1
    f1_score = metric_f1.compute(predictions=predictions, references=labels)

    # Cargar métrica recall
    metric_recall = evaluate.load("recall")

    # Calcular la métrica recall
    recall_score = metric_recall.compute(predictions=predictions, references=labels)

    # Cargar métrica Accuracy
    metric_accuracy = evaluate.load("accuracy")

    # Calcular la métrica Accuracy
    accuracy_score = metric_accuracy.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy_score['accuracy'], "f1": f1_score['f1'], "recall": recall_score['recall']}

In [ ]:
# Cargar el modelo DistilBERT pre-entrenado y el tokenizer
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.we

In [ ]:
# Convierta el marco de datos de python en un conjunto de datos de Huggingface
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)
hg_val_data = Dataset.from_pandas(val_data)

In [ ]:
# Función para tokenizar datos
def tokenize_dataset(data):
    return tokenizer(data["review"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")

# Tokenize el conjunto de datos
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)
dataset_val = hg_val_data.map(tokenize_dataset)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./distilbert/",
    logging_dir='./distilbert/logs',
    logging_strategy='epoch',
    logging_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [ ]:
# entrenar al modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall
1,0.482700,0.442909,0.828000,0.893564,0.947507
2,0.399100,0.450176,0.828000,0.890306,0.916010


TrainOutput(global_step=2000, training_loss=0.4408822021484375, metrics={'train_runtime': 117.9754, 'train_samples_per_second': 67.811, 'train_steps_per_second': 16.953, 'total_flos': 66233699328000.0, 'train_loss': 0.4408822021484375, 'epoch': 2.0})

In [ ]:
# Evaluar el conjunto de prueba
result_test = trainer.evaluate(dataset_test)
pd.DataFrame(result_test.items(), columns=["name", "value"])

,name,value
0,eval_loss,0.422260
1,eval_accuracy,0.826000
2,eval_f1,0.893773
3,eval_recall,0.938462
4,eval_runtime,1.804700
5,eval_samples_per_second,277.060000
6,eval_steps_per_second,69.265000
7,epoch,2.000000


#### c) Compara los resultados e incluye tus comentarios.

Comparando los resultados de los dos enfoques utilizados:

Utilizando un modelo predeterminado de HF de manera directa:

- Accuracy: 0.724
- F1-score: 0.8078
- Recall: 0.7436

Utilizando un ajuste de parámetros (fine-tuning) con el modelo Transformer DistilBERT:

- Accuracy: 0.826
- F1-score: 0.8938
- Recall: 0.9385

Se observa que el enfoque de fine-tuning con el modelo DistilBERT logra mejores resultados en todas las métricas de desempeño. El accuracy aumenta del 72.4% al 82.6%, el F1-score mejora del 80.8% al 89.4%, y el recall aumenta del 74.4% al 93.8%.

Esto indica que el ajuste de parámetros (fine-tuning) permite adaptar el modelo preentrenado a las características específicas de los datos de entrenamiento, lo cual resulta en una mejora significativa en la capacidad del modelo para clasificar los comentarios correctamente en diferentes clases de sentimientos. Al ajustar los pesos del modelo durante el proceso de entrenamiento, se logra una mejor captura de las relaciones y patrones relevantes presentes en los datos de entrenamiento.

Es importante tener en cuenta que el enfoque de fine-tuning requiere más tiempo de entrenamiento y recursos computacionales en comparación con el uso directo de un modelo predeterminado de HF. Sin embargo, los resultados demuestran que vale la pena realizar el ajuste de parámetros si se busca obtener un mejor desempeño en la tarea de análisis de sentimiento.

En resumen, el enfoque de fine-tuning con el modelo Transformer DistilBERT proporciona una mejora significativa en la precisión, el F1-score y el recall en comparación con el uso directo de un modelo predeterminado. Esto destaca la importancia de adaptar los modelos preentrenados a los datos específicos de la tarea para obtener un mejor rendimiento en aplicaciones de análisis de sentimiento.

#### 4. Utiliza el modelo Transformer pre-entrenado para diferentes idiomas llamado Helsinki-NLP de HF, para hacer una traducción del Inglés al Español de los 5000 comentarios. Observa algunos de los comentarios traducidos y comenta qué tan bien consideras dichas traducciones. Por el momento es un simple comentario subjetivo, aunque existen métricas y bases de datos para evaluar el desempeño de un Trasnformer.

NOTA: Guarda el archivo de los comentarios traducidos para facilitar su uso en las siguientes preguntas de esta actividad.

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tmpdfi = X.values.tolist()     # lista con los 748 registros.
newdfi=[]

for text in tmpdfi:
  if len(text) > 400:
            text = text[:400]
  newdfi.append(str(text))

print(f'Tamaño de la lista {len(newdfi)}')

Tamaño de la lista 5000


In [ ]:
# Loading the tokenizer:

encoding = tokenizer(newdfi,
                     padding=True,
                     truncation=False,
                     return_tensors='pt',   # probabilities
                     )

In [ ]:
pt_model = AutoModelForSeq2SeqLM.from_pretrained(model_name) # recuerda que cada vez que cargas algún modelo sube algo la RAM

In [ ]:
X_es=[]

for i in range(len(newdfi)):
   input_ids = tokenizer.encode(newdfi[i], return_tensors="pt")
   outputs = pt_model.generate(input_ids, max_length=512)
   decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
   X_es.append(decoded)

In [ ]:
print(f'Tamaño de la lista traducida: {len(X_es)}')

Tamaño de la lista traducida: 5000


In [ ]:
#Muestra del traductor
for i in range(0, 5):
    print(X_es[i])

Creo que es más caro que la farmacia que sólo cuesta 27usd.So usted tiene una buena elección si usted quiere, pero
Cuando vi esto en Amazon, lo puse en mi lista de deseos para que pudiera encontrarlo de nuevo. Sabía que sería el próximo regalo de bebé que iba a comprar. La oportunidad surgió recientemente y tan pronto como leí la invitación de la ducha, puse mi pedido. Esto se ve tan cómodo. Me encanta que tiene una segunda cubierta que le permite crecer con el niño. Tantos artículos diseñados para un bebé se utilizan sólo por un tiempo tan corto, pero esto debe ser capaz de ser utilizado por mucho más tiempo. He dado esto a la madre-a-ser un
Realmente nos gustan estos valances. Tienen personajes tan lindos en ellos. Les recomendamos mucho! La única caída es tener que planchar después de que salen de la secadora, pero vale la pena!
No hay luz que emita de la luz nocturna. Lo pintaron de un color sólido y no dejaron una abertura en la espalda. Terminarás devolviéndola.
Realmente esperab

Se utilizó el modelo Helsinki-NLP, un modelo Transformer pre-entrenado para la traducción de inglés a español, para traducir los 5000 comentarios de inglés al español. Se seleccionó el modelo "Helsinki-NLP/opus-mt-en-es" y se utilizó el tokenizer correspondiente para preparar los datos antes de la traducción.

Al revisar algunos ejemplos de los comentarios traducidos, se observa que en general las traducciones son comprensibles y capturan en su mayoría el significado general de los comentarios originales. Sin embargo, también se pueden identificar algunas limitaciones y errores en las traducciones, como errores gramaticales, falta de coherencia en algunas frases y la omisión o mala traducción de ciertas expresiones idiomáticas.

Es importante tener en cuenta que la calidad de las traducciones puede variar dependiendo de la complejidad y contexto de los textos originales, así como de las características específicas de los idiomas en cuestión. Los modelos de traducción automática, aunque han mejorado significativamente en los últimos años, todavía pueden presentar dificultades en la comprensión de matices y expresiones culturales.

En conclusión, el modelo Helsinki-NLP utilizado para la traducción de inglés a español proporciona traducciones comprensibles en su mayoría, pero aún existen áreas de mejora. Si se requiere una traducción precisa y de alta calidad, especialmente para textos más complejos o sensibles al contexto, es recomendable contar con la revisión humana o utilizar servicios de traducción profesionales. No obstante, los modelos de traducción automática como el utilizado en este ejercicio son útiles para obtener una idea general del contenido y facilitar la comprensión en situaciones menos críticas o informales.

Existen pocos modelos pre-entrenados con corpus en Español, pero dos de los más conocidos son
BETO ( https://github.com/dccuchile/beto ) y MarIA ( https://arxiv.org/abs/2107.07253 ). En
particular para análisis de sentimiento se puede utilizar alguno de los modelos derivados de BETO:

#### 5. Utiliza el modelo Transformer “pysentimiento”, basado en BETO y este a su vez basado en BERT, para llevar a cabo un análisis de sentimiento (sentiment analysis) y hacer la predicción de los comentarios traducidos al español. Incluye tus comentarios del resultado obtenido.

In [ ]:
file_name = 'amazonbaby5000_es.txt'
with open(file_name, 'r', encoding='utf-8') as file:
    content = file.read()
X_es = content.split('\n')

In [ ]:
#Dataframe de los comentarios en español
df_es = pd.DataFrame(X_es, columns=['review'])
df_es["label"] = Y
df_es

,review,label
0,Creo que es más caro que la farmacia que sólo ...,0
1,"Cuando vi esto en Amazon, lo puse en mi lista ...",1
2,Realmente nos gustan estos valances. Tienen pe...,1
3,No hay luz que emita de la luz nocturna. Lo pi...,0
4,Realmente esperaba que esto fuera un dispensad...,0
...,...,...
4995,Me gusta que este portador es como el Moby en ...,1
4996,La caja estaba dañada a la llegada. Tenía mied...,1
4997,Comprado para la graduación. Rec'd en 2 días c...,1
4998,Para todos los comentarios que dijeron que est...,0


In [ ]:
#Conteo de los valores de las etiquetas
df_es['label'].value_counts()

1    3840
0    1160
Name: label, dtype: int64

In [ ]:
# Conjunto de entrenamiento
train_data = df_es.sample(frac=0.8, random_state=42)

# Conjunto de prueba
temp = df_es.drop(train_data.index)

val_data = temp.sample(frac=0.5, random_state=42)

test_data = temp.drop(val_data.index)

print(f'El conjunto de entrenamiento tiene {len(train_data)} registros.')
print(f'El conjunto de validación tiene {len(val_data)} registos.')
print(f'El conjunto de prueba tiene {len(test_data)} registos.')

El conjunto de entrenamiento tiene 4000 registros.
El conjunto de validación tiene 500 registos.
El conjunto de prueba tiene 500 registos.


In [ ]:
model_name = "pysentimiento/robertuito-sentiment-analysis"

# Tokenizer de un modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Convierta el marco de datos de python en un conjunto de datos de Huggingface
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)
hg_val_data = Dataset.from_pandas(val_data)

In [ ]:
# Función para tokenizar datos
def tokenize_dataset(data):
    return tokenizer(data["review"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")

# Tokenize el conjunto de datos
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)
dataset_val = hg_val_data.map(tokenize_dataset)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# carga del modelo
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./pysentimiento/",
    logging_dir='./pysentimiento/logs',
    logging_strategy='epoch',
    logging_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [ ]:
# entrenar al modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall
1,0.134100,1.101790,0.804000,0.874359,0.848259
2,0.285800,1.127758,0.822000,0.888889,0.885572


TrainOutput(global_step=2000, training_loss=0.20996700286865233, metrics={'train_runtime': 180.7188, 'train_samples_per_second': 44.268, 'train_steps_per_second': 11.067, 'total_flos': 131555527680000.0, 'train_loss': 0.20996700286865233, 'epoch': 2.0})

In [ ]:
# Evaluar el conjunto de prueba
result_test = trainer.evaluate(dataset_test)
pd.DataFrame(result_test.items(), columns=["name", "value"])

,name,value
0,eval_loss,0.910799
1,eval_accuracy,0.826000
2,eval_f1,0.887160
3,eval_recall,0.914439
4,eval_runtime,3.294900
5,eval_samples_per_second,151.750000
6,eval_steps_per_second,37.937000
7,epoch,2.000000


En el análisis de sentimiento utilizando el modelo "pysentimiento" basado en BETO, se observa que el modelo logra una precisión del 82.6% en el conjunto de prueba, con un puntaje F1 del 88.7% y un recall del 91.4%. Estas métricas indican que el modelo es capaz de realizar una clasificación razonablemente precisa de los comentarios en términos de sentimiento que esta traducido en español.

Es importante tener en cuenta que el rendimiento del modelo puede variar según el conjunto de datos utilizado y la complejidad de los comentarios. En este caso, los comentarios son traducciones del inglés al español, lo que podría afectar la precisión de la clasificación de sentimientos, ya que las traducciones pueden introducir ruido o pérdida de información.

En conclusión, el modelo "pysentimiento" basado en BETO ofrece resultados prometedores en el análisis de sentimiento de los comentarios en español. Sin embargo, se recomienda realizar ajustes adicionales y considerar otras técnicas, como el ajuste fino y la optimización de hiperparámetros, para mejorar aún más el rendimiento del modelo en este dominio específico.


#### 6. Resume los resultados obtenidos en esta actividad de manera tabular e incluye tus comentarios finales.

NOTA: A partir de esta actividad, no dudes en experimentar con otros modelos Transformer preentrenados tanto para traducción al español, como para análisis de sentimiento.

En esta actividad se llevaron a cabo diferentes tareas relacionadas con el análisis de sentimiento en comentarios. Se trabajó con un conjunto de datos de 5000 comentarios en inglés, y se utilizaron varios modelos y técnicas para realizar el análisis.

En primer lugar, se entrenó el modelo Roberta Base Sentiment, este se implementó de manera directa con el que no obtuvo el mejor rendimiento en la clasificación de sentimientos, el resultado de las métricas son: precisión del 72%, un puntaje F1 del 81% y un recall del 74%.

Posteriormente, se exploró el uso del modelo Distil Bert para el análisis de sentimiento, con este se efectuó un ajuste de parámetros (fine-tuning) en Distil Bert, se obtuvieron las siguientes métricas: precisión del 82%, un puntaje F1 del 89% y un recall del 94%. Se puede ver que el modelo obtuvo un mejor rendimiento en comparación con el modelo Roberta Base Sentiment.

Se hizo una comparación de los resultados obtenidos entre el modelo Roberta Base Sentiment y Distil Bert, confirmando que el modelo Distil Bert superó al modelo Roberta Base Sentiment en términos de rendimiento en la tarea de análisis de sentimiento directa.

Luego, se utilizó el modelo Helsinki-NLP para traducir los comentarios en inglés al español, obteniendo una lista de 5000 comentarios traducidos.

Por último, se aplicó el modelo pysentimiento basado en BETO para hacer el análisis de sentimiento en los comentarios traducidos al español. Este modelo demostró un buen rendimiento, con una precisión del 83%, un puntaje F1 del 89% y un recall del 91% en el conjunto de prueba.

En resumen, se experimentó con diferentes modelos y técnicas para el análisis de sentimiento en comentarios. El modelo Roberta Base Sentiment y el modelo pysentimiento mostraron resultados sólidos en sus respectivas tareas, mientras que el modelo Distil Bert presentó un rendimiento inferior.

<center>

|         Actividad       |    Resultado  sobre el conjunto de prueba     |
|-------------------------|-----------------------------------------------|
| Total de comentarios    | 5,000                                         |
| Roberta Base Sentiment  | Accuracy: 72% <br> F1: 81% <br> Recall: 74%   |
| Distil BERT             | Accuracy: 82% <br> F1: 89% <br> Recall: 94%   |
| Helsinki NLP            | Traducción de comentarios de ingles a español |
| Pysentimiento           | Accuracy: 83% <br> F1: 89% <br> Recall: 91%   |

</center>